In [588]:
import re
import numpy as np
import pandas as pd
ids_CDLI = pd.read_csv('drehem_p_ids.txt', sep=" ", header=None, names=["CDLI"], dtype='str')
ids_BDTNS = pd.read_fwf('query_cat_17_09_6-194802.txt', sep=" ", header=None, names=["BDTNS", "CDLI"])
BDTNS = list(ids_BDTNS['CDLI'].str[1:])
CDLI = list(ids_CDLI['CDLI'])
#finds ids both in BDTNS and CDLI data
overlapping_ids = list(set(BDTNS) & set(CDLI))

In [589]:
#opens first text, TODO: Open and read more txts, not just group1
f = open('ur3_0.txt', 'r')
content = f.read().split("&P")[1:]
#content = f1.read().split("&P")[1:] + f2.read().split("&P")[1:] + f3.read().split("&P")[1:] + f4.read().split("&P")[1:]"""
#f1 = open('group1.txt', 'r')
#f2 = open('group2.txt', 'r')
#content = f1.read().split("&P")[1:] + f2.read().split("&P")[1:]
commodities = {}
#sets up commodities dictionary, where keys=pnumbers, and values=lematized lines with ;n
for c in (content):
    c_id = re.findall("(\d+) =",c)[0]
    if (c_id in overlapping_ids):
        text = re.findall("(\. \d)(.+?)(\\n)",c)
        text_lem = re.findall("(#lem: n;)(.+?)(\\n)",c)
        commodities[c_id] = list(zip(text, text_lem))


In [590]:
#original, unfiltered texts making everything worker, and taking out some filtered words from the original commodities dicitonary
#breaks each key.value into a list of values of items being exchanged
items = []
filtered1 = [("giŋ", "a unit of weight"), ('mana', 'a unit of weight'),('adda','corpse'),('sila', 'a unit of capacity'), ('lal', 'small'), ('šanabi', 'two-thirds'), ('a', 'arm'), ('bar', 'outside'), ('da', 'side board '),('DU', 'delivery'),('dub', 'smash'), ('dumu', 'worker'), ('e', 'leave'), ('ešeš', 'festival'), 
             ('gun', 'load'), ('hi', 'mix '), ('amar', 'young'), ('babbar', 'white'), ('gunu', 'speckled'), ('gur', 'unit of capacity'),('mu', 'name'), ('munus', 'woman'), ('niga', 'fattened'),('nita', 'male')
            ,('sag', 'fine'), ('tur', 'small'), ('uš', 'die'), ('ŋar', 'put'), ('šeš', 'brother'), ('ŋal', '⟦be⟧ to be (there'), ('ud', 'sun')]
text_0 = [("giŋ", "a unit of weight"),('zal', 'pass time'), ('adda','corpse'), ('lal', 'small'), ('sila', 'a unit of capacity'), ('šanabi', 'two-thirds'), ('amar', 'young'),
         ('gur', 'unit of capacity'), ('sag', 'fine')]
for c in commodities.keys():
    items_c = []
    for e in commodities[c]:
        lem = str(e[1])
        lem = re.sub("(person)|( male)|(child)", "worker", lem)
        lem = re.sub("~","", lem)
        #text = re.findall("\[(.*?)\]", lem)
        text = re.findall("(\w*)\[(.*?)\]", lem)
        for t in text:
            if (t=="unit"):
                text.remove(t)
            elif (t in filtered1):
                text.remove(t)
        if (len(text)>1) and (text[0] in text_0):
            items_c.append(text[1])
        elif len(text)==0:
            items_c.append(text)
        else:
            items_c.append(text[0])        
    items.append(items_c)

In [591]:
d = {'ip': list(commodities.keys()), 'commodities': items}
comm_df = pd.DataFrame(data=d)
comm_df["#commodities"] = comm_df["commodities"].apply(lambda x:len(x))
comm_df.head()

,ip,commodities,#commodities
0,100041,"[(udu, sheep)]",1
1,100189,"[(udu, sheep), (sila, lamb)]",2
2,100190,"[(sila, lamb), (sila, lamb), (u, sheep)]",3
3,100191,"[(mašgal, billy goat)]",1
4,100211,"[(udu, sheep), (udu, sheep), (mašgal, billy go...",6


In [592]:
test = comm_df.head(100)
titles = []
for c in list(comm_df['commodities']):
    if [] in c:
        c = [x for x in a if x != [1, 1]]
    titles = titles + (list(set(c) - set(titles)))

In [585]:
for t in titles:
    comm_df["% " + str(t)] = comm_df['commodities'].map(lambda a: a.count(t)*100)/comm_df['#commodities']
comm_df = comm_df.drop(["% 2","% 3", "% 1", "% 4"], axis = 1).set_index("ip")
#comm_df = comm_df.rename(index=str, columns={"#commodities":"#_of_commodities","commodities":"commodities_list", "comm_string":"commodities_string"})

In [587]:
#comm_df[comm_df["#_of_commodities"]==0]
#comm_df[comm_df["commodities_string"]=='[]']
comm_df1 = comm_df[comm_df['commodities'].apply(lambda x: (('ud', 'sun') in x)==True)]
comm_df1.index
#comm_df2 = comm_df.dropna()
#list(np.sort(comm_df.columns.unique()))
#comm_df
#('100040' in overlapping_ids)
#('100230' in overlapping_ids)

Index(['104285', '117320'], dtype='object', name='ip')

In [531]:
#text simply keeping first word in each list, filter out units/twothirds?
#['worker', 'sun', 'barley', 'milk'] 100014
items = []
filtered_out = ["two-thirds", "barley", "small","sun", "rare", "fattened","white"]
for c in commodities.keys():
    items_c = []
    for e in commodities[c]:
        lem = str(e[1])
        lem = re.sub("(person)|(male)|(child)", "worker", lem)
        lem = re.sub("~", "", lem)
        text = re.findall("\[(.*?)\]", lem)
        for t in text:
            if (t in filtered_out):
                text.remove(t)
        if len(text)==0:
            items_c.append(text)
        else:
            items_c.append(text)
    items.append(items_c)

In [223]:
x = [(0,1),(12,2)]
x[0][0]

0

In [8]:
"""comm_df['comm_string'] = comm_df['commodities'].astype(str).apply(lambda x: x[1:-1].replace(",", "").replace("'", ""))
tidy_format = comm_df['comm_string'].str.split(expand=True).stack().reset_index(level=1)
unique_comm = tidy_format[0].unique()"""

In [ ]:
7. 2(u) 5(diš) guruš u₄ 1(diš)-še₃ 
#lem: n; n; ŋuruš[male]; ud[sun]; n

1. 1(geš₂) 1(u) 3(diš) gu₄ ab₂ hi-a 
#lem: n; n; n; gud[ox]; ab[cow]; hi[mix]

1. 1(geš₂) 1(u) 3(diš) gu₄ ab₂ hi-a 
#lem: n; n; n; gud[ox]; ab[cow]; hi[mix]

person, child, male, female = worker
anything w garden = garden